In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import scipy as sp
import matplotlib.pyplot as plt
import glob
%gui qt5
import napari
import tifffile
import plotly.graph_objs as go

In [2]:
viewer = napari.Viewer()

In [3]:
df = pd.DataFrame()
for f in glob.glob('*.csv'):
    tdf = pd.read_csv(f)
    tdf['File'] = f
    df = pd.concat([df, tdf])

In [4]:
df['gene'] = df['File'].str.split('_').str[0]
df['channel'] = df['Label'].str.split(':').str[0].str[-1]
df['file'] = df['File'].str.split(':').str[0].str[0:-4]
df['time'] = df['File'].str.split('_').str[1]
df['time'] = df['time'].map({'homeo':'00dpa', '1dpa':'01dpa', '2dpa':'02dpa'})
df['SSlice'] = "S"+df['Slice'].astype(str)
df['timen'] = df['time'].str[0:2].astype(int)
df['region'] = df['Label'].str.split('_').str[2]

KeyError: 'File'

I found that the very first slice (which is entirely epithelium), sometimes has garbage on it that makes cells show up as positive when they really are not, so the simple fix is to simply throw out those cells.  It also messes up the normalization to come if they are present.

In [ ]:
df = df[df['Slice']>1]

# Normalize

Intensities across samples that are treated the same way are not similar.  Sif is present in all images yet varies significantly in its intensity levels.  This normalizes for each file both channels by finding the 30th percentile of intensity, and dividing by that amount.  The 90th percentile correction proved too sensitive.

In [ ]:
rdf = df[df['channel']=='1']
tdf = df[df['channel']=='2']

rdf['Mean2'] = list(tdf['Mean'])
rdf['IntDen2'] = list(tdf['IntDen'])

#normer = rdf.groupby(['file']).agg({'Mean':np.median, 
#                                   'Mean2':np.median}).reset_index().rename(columns={'Mean':'Median', 'Mean2':'Median2', 'file':'lfile'})
normer_low = rdf.groupby(['file']).agg({'Mean':(lambda x: np.percentile(x, 30)), 
                                   'Mean2':(lambda x:np.percentile(x,30))}).reset_index()
normer_high = rdf.groupby(['file']).agg({'Mean':(lambda x: np.percentile(x, 90)), 
                                   'Mean2':(lambda x:np.percentile(x,90))}).reset_index()
normer = normer_low.merge(normer_high, on='file').rename(columns={'Mean_x':'LowMean', 'Mean2_x':'LowMean2', 'Mean_y':'HighMean',
                                                        'Mean2_y':'HighMean2', 'file':'lfile'})

rdf = rdf.merge(normer, left_on='file', right_on='lfile')
#rdf['Mean'] = 100000*(rdf['Mean']-rdf['LowMean'])/(rdf['HighMean']-rdf['LowMean'])
#rdf['Mean2'] = 100000*(rdf['Mean2']-rdf['LowMean2'])/(rdf['HighMean2']-rdf['LowMean2'])
rdf['Mean'] = rdf['Mean']/rdf['LowMean'] * 10000
rdf['Mean2'] = rdf['Mean2']/rdf['LowMean2'] * 10000

rdf = rdf[(rdf['Mean']>0) & (rdf['Mean2']>0)]

rdf['LMean'] = np.log(0.0001+rdf['Mean'])
rdf['LMean2'] = np.log(0.0001+rdf['Mean2'])

In [ ]:
px.histogram(rdf[rdf['gene']=='agat-1'], x="LMean", facet_row='file', height=1200
                   )

In [ ]:
px.scatter(rdf, x='LMean', y='LMean2', hover_data=['X', 'Y', 'Slice', 'file'], facet_col='timen',
           facet_row='gene',  color='file', height=800)

In [ ]:
rdf.shape

This is messy, but it allows to see some broad trends.  More importantly it lets us click off individual files to see if they are different from the general trends.

In [ ]:
def clear_layers():
    for i in range(0, len(viewer.layers)):
        viewer.layers.remove(viewer.layers[0].name)

In [5]:
sdf = rdf.reset_index().reset_index()

f=go.FigureWidget(
    px.scatter(sdf, x='LMean', y='LMean2', hover_data=['file', 'Slice', 'X', 'Y', 'level_0'], 
               height=1200, log_x=True, log_y=True, color='file', facet_row='gene')
    )
def click_fn(trace, points, state):
    #print(f.data[points.trace_index]['customdata'][points.point_inds[-1]])
    if (len(points.point_inds)>0):
        clear_layers()
        idx = f.data[points.trace_index]['customdata'][points.point_inds[-1]][4]
        cur = rdf.iloc[idx]
        print([idx, points.point_inds[0]])
        
        img = tifffile.imread(cur['file']+'.tif')
        viewer.add_image(img, channel_axis=1, blending='additive')
        viewer.layers[-1].visible=False
        img = tifffile.imread(cur['file']+'_mask.tif')
        viewer.add_image(img, blending='additive', colormap='magenta')
        viewer.camera.center = [0,cur['Y'], cur['X']]
        viewer.camera.zoom=8
        viewer.dims.current_step = (cur['Slice']-1,0,0)
        
        viewer.layers[-1].contrast_limits=[0,4.0]
        viewer.layers[-3].contrast_limits=[0,15000]
        viewer.layers[-4].contrast_limits=[0,2000]
        #file = f.data[points.trace_index]['customdata'][points.point_inds[-1]][0]
        #show_image(file+'_processed.tif', viewer)


for s in f.data:
    s.on_click(click_fn)
#f.data[0].on_click(click_fn)

f

NameError: name 'rdf' is not defined

Although the above is clickable, it is not as informative as all spots are overlapping with each other.  The heatmap below shows trends better.

In [6]:
px.density_heatmap(rdf, x="LMean", y="LMean2",  
                   nbinsx=150, nbinsy=150, facet_row='gene', height=1200, facet_col='timen', #animation_frame='file',
                   color_continuous_scale=[(0, "blue"), (0.01, "green"), (0.04, 'yellow'), (1, "red")])

NameError: name 'rdf' is not defined

In [7]:
rdf.to_csv('HeatmapScatterPlots.csv')

NameError: name 'rdf' is not defined

Note that there are in prog-1 (and to some extent piwi) cells which show up on the diagonal.  Investigating these I found that these are NOT specifically labeled cells, but are gut/proto cells that are naturally autfluorescent in all channels and which for some reason show up a lot in some from prog-1 homeo.

# Classify using thresholds

In [8]:
def show_labeled_image(file):
    clear_layers()
    img = tifffile.imread(file)
    viewer.add_image(img, channel_axis=1)
    viewer.layers[-3].colormap='magenta'

    file = file[0:-4]

    tdf = rdf[(rdf['file']==file) & (rdf['Positive'])]
    pts = np.zeros([img.shape[0], img.shape[2], img.shape[3]])
    pts[tdf['Slice'].astype(int)-1, tdf['Y'].astype(int), tdf['X'].astype(int)] = 255
    viewer.add_image(pts, blending='additive')

    tdf = rdf[(rdf['file']==file) & (rdf['Positive2'])]
    pts = np.zeros([img.shape[0], img.shape[2], img.shape[3]])
    pts[tdf['Slice'].astype(int)-1, tdf['Y'].astype(int), tdf['X'].astype(int)] = 255
    viewer.add_image(pts, blending='additive')

    tdf = rdf[(rdf['file']==file) & (rdf['BothPositive'])]
    pts = np.zeros([img.shape[0], img.shape[2], img.shape[3]])
    pts[tdf['Slice'].astype(int)-1, tdf['Y'].astype(int), tdf['X'].astype(int)] = 255
    viewer.add_image(pts, blending='additive')
    
    viewer.layers[-3].colormap='magenta'
    viewer.layers[-2].colormap='green'
    viewer.layers[-1].colormap='gray'

These thresholds were settled upon through a series of trial and errors:  trying a threshold and then looking at the results by clicking on points and looking at images.  

In [9]:
sif = np.exp(10.5)
prog1 = np.exp(12.2)
piwi = np.exp(10.6)
agat3 = np.exp(11.0)
agat1 = np.exp(11.0)

#sif = np.exp(10.4)
#prog1 = np.exp(10.8)
#piwi = np.exp(11)
#agat3 = np.exp(11.2)
#agat1 = np.exp(11.4)


mapper = {'agat-1':agat1, 'agat-3':agat3, 'piwi':piwi, 'prog-1':prog1}

rdf['Cutoff'] = rdf['gene'].map(mapper)
rdf['Positive'] = rdf['Mean'] > sif
rdf['Positive2'] = rdf['Mean2'] > rdf['Cutoff']
rdf['BothPositive'] = (rdf['Positive']) & (rdf['Positive2'])

NameError: name 'rdf' is not defined

In [10]:
agged = rdf.groupby(['gene', 'timen', 'file']).agg({'Positive':np.sum, 'Positive2':np.sum, 'BothPositive':np.sum})
agged['FracDoublePositive'] = agged['BothPositive']/agged['Positive'] * 100


f=go.FigureWidget(
    px.box(agged.reset_index(), x='gene', y='FracDoublePositive', color='timen', points='all', hover_data=['file'])
    )
     
def click_fn(trace, points, state):
    
    if (len(points.point_inds)>0):
        file = f.data[points.trace_index]['customdata'][points.point_inds[-1]][0]
        print(file)
        show_labeled_image(file+'.tif')

for a in range(0,len(f.data)):
    f.data[a].on_click(click_fn)

f

NameError: name 'rdf' is not defined

In [11]:
agged = rdf.groupby(['gene', 'time', 'file']).agg({'Positive':np.sum, 'Positive2':np.sum, 'BothPositive':np.sum}).sort_values(by='time')
agged['FracDoublePositive'] = agged['BothPositive']/agged['Positive'] * 100
px.box(agged.reset_index(), x='gene', y='FracDoublePositive', color='time', points='all', hover_data=['file'])

NameError: name 'rdf' is not defined

In [12]:
agged = rdf.groupby(['gene', 'time', 'file']).agg({'Positive':np.mean, 'Positive2':np.mean}).reset_index().sort_values(by='time')
px.box(agged, x='gene', y='Positive2', color='time', points='all', hover_data=['file'], title='Gene positive')

NameError: name 'rdf' is not defined

This recapitulates what we have observed before:  at homeostasis about 1/6 of cells are piwi positive.  Also for all genes there is an enrichment of cells after amputation.  Also it is good to note that since BOTH agat-1 and agat-3 colocalize with sif, they should colocalize with each other so agat-1 and agat-3 should be giving the same results here.

In [13]:
agged = rdf.groupby(['gene', 'time', 'file']).agg({'Positive':np.mean, 'Positive2':np.mean}).reset_index().sort_values(by='time')
px.box(agged, x='gene', y='Positive', color='time', points='all', hover_data=['file'], title='Sif fraction positive')

NameError: name 'rdf' is not defined

Here we are only looking at sif, but we are breaking it out by the secondary gene.  We see that the results are the same irrespective of the gene, which should be the case.